In [1]:
from mb.gpt.models.connecter_type1 import BasicLLM
from mb.gpt.models.vlm_encoder_clip import VlmEncoderTest
from mb.gpt.models.text_encoder_smalvlm import TextEncoderTest

In [3]:
vlm_encoder = VlmEncoderTest(128,128)
text_encoder = TextEncoderTest(256,64)
model = BasicLLM(vocab_size=1000, embedding_dim=256, vlm_emb_dim=128, text_emb_dim=64)
model

BasicLLM(vocab_size=1000, embedding_dim=256)

In [9]:
for module in model.modules():
    print(module)

BasicLLM(vocab_size=1000, embedding_dim=256)
Embedding(1000, 256)
Linear(in_features=192, out_features=256, bias=True)


In [26]:
import torch

data1 = torch.randn(10, 128)  # Simulated VLM embedding
data2 = torch.randn(10, 64)   # Simulated text embedding


In [27]:
data1[0,:10]

tensor([-0.1996, -0.8266, -1.8745, -0.5909,  0.0975,  0.5615, -0.3793,  1.0049,
         0.4421,  0.0572])

In [28]:
output = model(data1, data2, torch.randint(0, 1000, (10, 20)))  # Simulated input_ids

In [29]:
output.shape

torch.Size([10, 20, 256])

In [30]:
output.backward(torch.rand_like(output))

In [31]:
data1[0,:10]

tensor([-0.1996, -0.8266, -1.8745, -0.5909,  0.0975,  0.5615, -0.3793,  1.0049,
         0.4421,  0.0572])